In [1]:
import boto3
import pandas as pd
import numpy as np
from copy import deepcopy as cp
import json
from pprint import pprint

In [2]:
aws_profile = 'peter-personal'
session = boto3.Session(profile_name='peter-personal')

In [3]:
# Read in the configuration Files
with open('FlowerTypes.json') as json_data:
    flower_types_dict = json.load(json_data)

with open('OrderFlowers.json') as json_data:
    order_flowers_dict = json.load(json_data)
    
with open('OrderFlowersBot.json') as json_data:
    order_flowers_bot_dict = json.load(json_data)

In [4]:
# Make Bot
model_builder = session.client('lex-models')
model_builder.put_slot_type(**flower_types_dict)
model_builder.put_intent(**order_flowers_dict)
model_builder.put_bot(**order_flowers_bot_dict)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '779',
   'content-type': 'application/json',
   'date': 'Tue, 20 Mar 2018 19:01:59 GMT',
   'x-amzn-requestid': '2b016b73-2c71-11e8-b698-cf5427835c9b'},
  'HTTPStatusCode': 200,
  'RequestId': '2b016b73-2c71-11e8-b698-cf5427835c9b',
  'RetryAttempts': 0},
 'abortStatement': {'messages': [{'content': "Sorry, I'm not able to assist at this time",
    'contentType': 'PlainText'}]},
 'checksum': 'a587f117-18ae-4843-b0e8-d3607b4a9261',
 'childDirected': False,
 'clarificationPrompt': {'maxAttempts': 2,
  'messages': [{'content': "I didn't understand you, what would you like to do?",
    'contentType': 'PlainText'}]},
 'createVersion': False,
 'createdDate': datetime.datetime(2018, 3, 20, 15, 1, 59, 944000, tzinfo=tzlocal()),
 'description': 'Bot to order flowers on the behalf of a user',
 'idleSessionTTLInSeconds': 600,
 'intents': [{'intentName': 'OrderFlowers', 'intentVersion': '$LATEST'}],
 'lastUpdat

In [5]:
lex_runtime = session.client('lex-runtime')

In [6]:
# Interact with Bot
lex_runtime = session.client('lex-runtime')
out = lex_runtime.post_text(botName = 'OrderFlowersBot',
                      botAlias = "$LATEST",
                      userId = "UserOne",
                      inputText = "i would like to order flowers" )
pprint(out)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '279',
                                      'content-type': 'application/json',
                                      'date': 'Tue, 20 Mar 2018 19:03:23 GMT',
                                      'x-amzn-requestid': '5d0d9de4-2c71-11e8-b202-c1f3d18f6e51'},
                      'HTTPStatusCode': 200,
                      'RequestId': '5d0d9de4-2c71-11e8-b202-c1f3d18f6e51',
                      'RetryAttempts': 0},
 'dialogState': 'ElicitSlot',
 'intentName': 'OrderFlowers',
 'message': 'What type of flowers would you like to order?',
 'messageFormat': 'PlainText',
 'slotToElicit': 'FlowerType',
 'slots': {'FlowerType': None, 'PickupDate': None, 'PickupTime': None}}
